<a href="https://colab.research.google.com/github/nicolez9911/colab/blob/main/02_ExerciseCatsandDogs-0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cats VS Dogs

Install latest version of kaggle library

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

1. Create a kaggle account and from your settings create a new API token to download a file called kaggle.json

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!ls ~/.kaggle

2. Go to this link :https://www.kaggle.com/c/dogs-vs-cats/data
and accept the terms of the competition

In [ ]:
!kaggle competitions download -c dogs-vs-cats

In [ ]:
!unzip -qq /content/dogs-vs-cats.zip

In [ ]:
!ls /content

In [ ]:
!unzip -qq /content/train.zip
!unzip -qq /content/test1.zip

**Copying images to training, validation, and test directories**

In [ ]:
import os, shutil, pathlib

original_dir = pathlib.Path("/content/train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

### Load Dataset

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)